In [ ]:
import requests
import pandas as pd
import datetime as dt
import json
import matplotlib.pyplot as plt

k = 123
#
base = "https://clarity-data-api.clarity.io/v1/measurements"

head = {
    'x-api-key':k,
    "Accept-Encoding":"gzip"
}

limit = 20000

In [ ]:
def linear_correction(raw_val):
    return (raw_val--4.933)/1.8766

def mlr_correction(raw_val,temp,rel_hum):
    return ( raw_val-(-75.2019) - (temp*0.8060) - (rel_hum*0.6382) )  / 1.8352

In [ ]:
##### HUMAN OUTPUT ######
#qc list
sensors = pd.read_csv("sensors.csv",parse_dates=['inst_date'],index_col=['id'])
outputfreq='hour' #other options 'hour' or 'min'

start = "2023-05-01 00:00:00"
end = "2023-05-02 00:00:00" #PH TIME


In [ ]:
end = pd.to_datetime(end) - dt.timedelta(hours=8) #conversion to GMT+0
start = pd.to_datetime(start) - dt.timedelta(hours=8) #conversion to GMT+0
master = pd.DataFrame()
no_temps = pd.DataFrame()
filename_date_format = "%d-%b-%Y %H_%M"

tempcol = 'characteristics.temperature.value'
rh_col = 'characteristics.relHumid.value'
pm25raw_col = 'characteristics.pm2_5ConcMass.raw'
calib_col = 'characteristics.pm2_5ConcMass.calibratedValue'

for index, row in sensors.iterrows():
    dev_id = row["dev_id"]
    
    s_name = row['short_name']
    
    params = {
    "code":dev_id,
    "outputFrequency":outputfreq,
    "startTime":start,
    "endTime":end,
    "limit":limit
    }
    try:
        print('getting results for',dev_id)
        result = requests.get(base,params=params,headers=head)
        a = result.json()
        df = pd.json_normalize(a)
        df['time'] = pd.to_datetime(df['time'])
        df = df.set_index('time')
        df = df.tz_convert("Asia/Manila")
        df = df.tz_localize(None)
        df = df.sort_index()

        cur = df[[tempcol,calib_col,rh_col,pm25raw_col]]
       
        mlr_colname = "pm2.5_MLR_" +s_name
        linear_colname = "pm2.5_linear_" + s_name
        cur[linear_colname] = linear_correction(cur[pm25raw_col])
        cur[mlr_colname] = mlr_correction(cur[pm25raw_col],cur[tempcol],cur[rh_col])

        cur = cur.rename(columns = {
            tempcol:"temp_"+s_name,
            rh_col:"rh_"+s_name,
            pm25raw_col:"pm2.5_raw_"+s_name,
            calib_col:"pm2.5_calib_"+s_name
        })
        
        cur_no_temp = cur[["pm2.5_calib_"+s_name]]
        
        master = master.join(cur,how="outer", lsuffix='a')
        no_temps = no_temps.join(cur_no_temp,how="outer",lsuffix='a')
    except KeyError:
        pass

In [ ]:
maxdf = pd.DataFrame()
maxdf['max_timestamp'] = master.idxmax(axis=0)
maxdf['max_value'] = master.max()

In [ ]:
summary = pd.DataFrame()
today = "2023-05-01"

am_start = "06:00"
am_end = "08:00"

noon_start = "11:00"
noon_end = "13:00"

pm_start = "16:00"
pm_end = "18:00"

# am_df = no_temps.loc[today+" "+am_start:today+" "+am_end]
# noon_df = no_temps.loc[today+" "+noon_start:today+" "+noon_end]
# pm_df = no_temps.loc[today+" "+pm_start:today+" "+pm_end]

am_df = master.loc[today+" "+am_start:today+" "+am_end]
noon_df = master.loc[today+" "+noon_start:today+" "+noon_end]
pm_df = master.loc[today+" "+pm_start:today+" "+pm_end]


summary['am_ave'] = am_df.sum() / 3
summary['noon_ave'] = noon_df.sum() / 3
summary['pm_ave'] = pm_df.sum() / 3
summary = summary.join(maxdf,how='outer')
summary

In [ ]:
dev_base = "https://clarity-data-api.clarity.io/v1/devices"

dev_head = {
    'x-api-key':k
}

result = requests.get(dev_base,headers=dev_head)
a = result.json()
df = pd.json_normalize(a)

status_df = df[['code','batteryStatus','sensorsHealthStatus','overallStatus']]
status_df

In [ ]:
no_temps.plot(figsize=(18,6))

In [ ]:
pd.merge(sensors[["dev_id","long_name"]], status_df, left_on='dev_id', right_on='code')